In [ ]:
!pip3 install selenium

In [ ]:
#Install Selenium package- 

In [ ]:
from selenium import webdriver
import time
import pandas as pd
import json

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
from   scipy   import   stats

import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import metrics 

In [ ]:
url=(r'https://www.linkedin.com/jobs/search?location=United%20States&geoId=103644278&locationId=&keywords=Data%20Analyst&f_TPR=r86400&trk=homepage-jobseeker_recent-search&position=1&pageNum=0')

In [ ]:
url

In [ ]:
from bs4 import BeautifulSoup as soup

In [ ]:
job_csv_file_path= "./job_csv/"
if not os.path.isdir(job_csv_file_path):
    os.mkdir(job_csv_file_path)
SLEEP_TIME = 5
chromedriver_exe = input("Provide path to the chromedriver executable")
browser = webdriver.Chrome(executable_path=chromedriver_exe)
browser.get(url)
i = 0
APPROX_MAX_COUNT = 100
count = 0
f_cnt = 0
while(count < APPROX_MAX_COUNT):
    try:
        browser.find_element_by_xpath("/html/body/div[1]/div/main/section[2]/button").click()
        time.sleep(SLEEP_TIME)
    except:
        pass
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        ul = browser.find_element_by_xpath("/html/body/div[1]/div/main/section[2]/ul")
        job_links = ul.find_elements_by_tag_name("li")
    except:
        pass
    curr_count = count
    job_list=[]
    for job_link in job_links[curr_count:]:
        if count > APPROX_MAX_COUNT:
            break
        job_data = {}
        #print(job_link)
        job_link.click()
        time.sleep(SLEEP_TIME)
        page = soup(browser.page_source,'lxml')
        try:
            job_data['job_title'] =page.find('h2',{'class':'top-card-layout__title'})
            job_data['company_name'] = page.find('span',{'class':'topcard__flavor'})
            job_data['location'] = page.find('span',{'class':'topcard__flavor topcard__flavor--bullet'})
            job_data['post_time'] = page.find('span',{'class':'posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata'})
            job_data['applicant_count'] = page.find('span',{'class':'num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet'})
            #job_data['summary'] = page.find('div',{'class':'show-more-less-html__markup'})
            job_data['salary'] = page.find("div",{"class":"salary compensation__salary"})
            job_criteria_ul = page.find('ul',{'class':'description__job-criteria-list'})
            job_criterion_list = job_criteria_ul.find_all('li')
            for job_criteria in job_criterion_list:
                job_criteria_field = job_criteria.find('h3',{'class':"description__job-criteria-subheader"})
                job_criteria_value = job_criteria\
                .find('span',{'class':"description__job-criteria-text description__job-criteria-text--criteria"})
                job_data[job_criteria_field.get_text().strip()] = job_criteria_value
            for k,v in job_data.items():
                if v:
                    job_data[k] = v.get_text().strip()
        except Exception as e:
            pass
            print(e)
            time.sleep(SLEEP_TIME)
        job_list.append(job_data)
        count+=1
    if job_list:
        df = pd.DataFrame(job_list)
        csv_path = f"{job_csv_file_path}/{f_cnt}.csv"
        f_cnt+=1
        df.to_csv(csv_path,index=False)
    
browser.quit()

In [ ]:
df = pd.DataFrame()
for f in os.listdir(job_csv_file_path):
    file_name = f"{job_csv_file_path}/{f}"
    #print(file_name)
    df_temp = pd.read_csv(file_name)
    df = pd.concat([df,df_temp],ignore_index=True, sort=False)
    

In [ ]:
pd.DataFrame.drop_duplicates(df)
df.head(10)

In [ ]:
# Clean locations
def split_location(e):
    us = set(["US","United States"])
    s_list = e.split(",")
    if len(s_list)==2 and not any([(s.strip() in us) for s in s_list]):
        return e
    else:
        return "Not Applicable,Not Applicable"
city_state = df["location"].apply(split_location)
city_state_df = city_state.str.split(pat=",",expand=True)
df["city"],df["state"] = city_state_df[0],city_state_df[1]

In [ ]:
# Clean post time
def post_time_to_days(e):
    hour_set = set(["hours","hour"])
    day_set = set(["days","day"])
    week_set = set(["week","weeks"])
    month_set = set(["month","months"])
    if type(e) is str:
        count,unit,rest =  e.split()
        if unit.lower() in hour_set:
            return f'{(int(count)/24):.2f}'
        elif unit.lower() in day_set:
            return count
        elif unit.lower() in week_set:
            return str(int(count)*7)
        elif unit.lower() in month_set:
            return str(int(count)*30)
        else:
            return None

df["posted_days"] = df["post_time"].apply(post_time_to_days)

In [ ]:
# Convert applicants to int
df["applicant_count_int"] =  df["applicant_count"].str.split(expand=True)[0]

In [ ]:
df

In [ ]:
#Loading

In [ ]:
!pip install sqlalchemy

In [ ]:
import platform
print(platform.node())

In [ ]:
pyodbc.drivers()

In [ ]:
from sqlalchemy import create_engine
import sqlalchemy 
import pyodbc
import pandas as pd
import socket

conn = sqlalchemy.create_engine(f'mssql+pyodbc://{socket.gethostname()}/Test?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server')
#df = pd.read_csv("./data.csv")

# quoted = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=(localhost,1433);DATABASE=Portfolio")
# engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

df.to_sql('JobData',con = conn,if_exists= 'append', index = False)

# result = engine.execute('SELECT COUNT(*) FROM [dbo].[JobDataTable]')
# result.fetchall()

In [ ]:
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost.com:1433;DATABASE=Portfolio;Trusted_Connection=yes;')


In [ ]:
!pip3 install pymssql

In [ ]:
import pymssql

In [ ]:
passwd = input("Enter password for SQL db")
conn = pymssql.connect(
    host=r'localhost',
    user=r'sa',
    password=passwd,
    database='Portfolio'
)

In [ ]:
cursor = conn.cursor(as_dict=True)

In [ ]:
cursor.execute('SELECT * from JobData')

In [ ]:
data = cursor.fetchall()
data_df = pd.DataFrame(data)

In [ ]:
data_df

In [ ]:
from sqlalchemy import event
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True
                
df.to_sql(JobData, engine, index=False, if_exists="append", schema="dbo")


In [ ]:
driver = ''
server = 'localhost'
database = 'Portfolio'
username = 'sa'
params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + passwd

In [ ]:
pyodbc.drivers()

In [ ]:
!pip3 install --upgrade pyodbc